### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.metrics import confusion_matrix
from sklearn import metrics
import sklearn
from sklearn.model_selection import StratifiedKFold
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Load the Computed Scores

In [2]:
scores = pd.read_csv('/content/drive/MyDrive/D/scores.csv')  #get the computed trolls scores
scores = scores.sample(frac=1).reset_index(drop=True)

### Functions

In [3]:
def label_by_score(df, threshold):   #function that get the data and classify the accounts based on the threshold
  check_predictions=[]
  
  for i in range(len(df)):
    if df.loc[i]['troll_score'] <= threshold:
      check_predictions.append(0)
    else:
      check_predictions.append(1)

  check_lab=df['label'].values

  acc=sklearn.metrics.accuracy_score(check_lab, check_predictions)      #accuracy
  prec = sklearn.metrics.precision_score(check_lab, check_predictions)  #precision
  rec = sklearn.metrics.recall_score(check_lab, check_predictions)      #recall
  f1 = sklearn.metrics.f1_score(check_lab, check_predictions)           #f1-score

  tn, fp, fn, tp = confusion_matrix(check_lab,check_predictions).ravel()
  fpr, tpr, thresholds = metrics.roc_curve(check_lab,check_predictions)
  auc = metrics.auc(fpr, tpr)  #auc 

  tpr = tp/float(tp+fn)
  tnr = tn/float(tn+fp)

  return (acc,prec,rec,f1,tpr, tnr, auc)

#function that returns the max accuracy, precsion, recall, tpr, tnr, f1, auc with their corresponding threshold
#threshold range between 0 and 1 with a step of 0.02
def get_all_values(scores):   
  threshold=0

  max_acc_thr=0
  max_prec_thr=0
  max_rec_thr=0
  max_tpr_thr=0
  max_tnr_thr=0
  max_f1_thr=0
  max_auc_thr=0


  max_acc=0
  max_prec=0
  max_rec=0
  max_tpr=0
  max_tnr=0
  max_f1=0
  max_auc=0

  while threshold <=1:
    acc,prec,rec,f1,tpr, tnr, auc = label_by_score(scores, threshold)

    if acc >= max_acc:
      max_acc=acc
      max_acc_thr = threshold

    if prec >= max_prec:
      max_prec=prec
      max_prec_thr = threshold

    if rec >= max_rec:
      max_rec=rec
      max_rec_thr = threshold
    
    if tpr >= max_tpr:
      max_tpr=tpr
      max_tpr_thr = threshold
    
    if tnr >= max_tnr:
      max_tnr=tnr
      max_tnr_thr = threshold

    if f1 >= max_f1:
      max_f1=f1
      max_f1_thr = threshold
    
    if auc >= max_auc:
      max_auc=auc
      max_auc_thr = threshold
    
    threshold = threshold + 0.02
  
  return (max_acc_thr, max_prec_thr, max_rec_thr, max_tpr_thr, max_tnr_thr, max_f1_thr, max_auc_thr)

### Threshold Computation

In [4]:
X = scores['troll_score'].values
y = scores['label'].values

In [5]:
#calculating the best acc, prec, .. and saving the performance with stratified k-fold
#metrics
Acc_a =[]
Pre_a =[]
Rec_a = []
Auc_a = []
F1_a =[]
TPR_a = []
TNR_a = []


Acc_auc =[]
Pre_auc =[]
Rec_auc = []
Auc_auc = []
F1_auc =[]
TPR_auc = []
TNR_auc = []


Acc_prec=[]
Pre_prec =[]
Rec_prec = []
Auc_prec = []
F1_prec =[]
TPR_prec = []
TNR_prec = []

Acc_rec=[]
Pre_rec =[]
Rec_rec = []
Auc_rec = []
F1_rec =[]
TPR_rec = []
TNR_rec = []

Acc_f1=[]
Pre_f1 =[]
Rec_f1 = []
Auc_f1 = []
F1_f1 =[]
TPR_f1 = []
TNR_f1 = []


Acc_tpr=[]
Pre_tpr=[]
Rec_tpr = []
Auc_tpr = []
F1_tpr =[]
TPR_tpr = []
TNR_tpr = []


Acc_tnr=[]
Pre_tnr=[]
Rec_tnr = []
Auc_tnr = []
F1_tnr =[]
TPR_tnr = []
TNR_tnr = []


Recalls=[]
Precisions=[]
AUCs=[]
Accs=[]
TNRs=[]
F1s=[]

# Instantiate the cross validator
skf = StratifiedKFold(n_splits=10, shuffle=True)
# Loop through the indices the split() method returns
for index, (train_indices, val_indices) in enumerate(skf.split(X, y)):
      # Generate batches from indices
      x_train, x_test = X[train_indices], X[val_indices]
      y_train, y_test = y[train_indices], y[val_indices]

      df_train = pd.DataFrame(columns=['troll_score','label'])
      df_train['troll_score']=x_train
      df_train['label']=y_train

      max_acc_thr, max_prec_thr, max_rec_thr, max_tpr_thr, max_tnr_thr, max_f1_thr, max_auc_thr = get_all_values(df_train) #get the best performance threshold for the train set

      Recalls.append(max_rec_thr)
      Precisions.append(max_prec_thr)
      AUCs.append(max_auc_thr)
      Accs.append(max_acc_thr)
      TNRs.append(max_tnr_thr)
      F1s.append(max_f1_thr)

      df_test=pd.DataFrame(columns=['troll_score','label'])
      df_test['troll_score']=x_test
      df_test['label']=y_test

      # performance with thresholf of best accuracy
      acc,prec,rec,f1,tpr, tnr, auc = label_by_score(df_test,max_acc_thr)  
      #save the performance
      Acc_a.append(acc)
      Pre_a.append(prec)
      Rec_a.append(rec)
      Auc_a.append(auc)
      F1_a.append(f1)
      TPR_a.append(tpr)
      TNR_a.append(tnr)
      
    # performance with thresholf of best precision
      acc,prec,rec,f1,tpr, tnr, auc = label_by_score(df_test,max_prec_thr)

      Acc_prec.append(acc)
      Pre_prec.append(prec)
      Rec_prec.append(rec)
      Auc_prec.append(auc)
      F1_prec.append(f1)
      TPR_prec.append(tpr)
      TNR_prec.append(tnr)

      # performance with thresholf of best recall
      acc,prec,rec,f1,tpr, tnr, auc = label_by_score(df_test,max_rec_thr)

      Acc_rec.append(acc)
      Pre_rec.append(prec)
      Rec_rec.append(rec)
      Auc_rec.append(auc)
      F1_rec.append(f1)
      TPR_rec.append(tpr)
      TNR_rec.append(tnr)

      # performance with thresholf of best tpr (recall)
      acc,prec,rec,f1,tpr, tnr, auc = label_by_score(df_test,max_tpr_thr)

      Acc_tpr.append(acc)
      Pre_tpr.append(prec)
      Rec_tpr.append(rec)
      Auc_tpr.append(auc)
      F1_tpr.append(f1)
      TPR_tpr.append(tpr)
      TNR_tpr.append(tnr)

      # performance with thresholf of best TNR (true negative rate)

      acc,prec,rec,f1,tpr, tnr, auc = label_by_score(df_test,max_tnr_thr)

      Acc_tnr.append(acc)
      Pre_tnr.append(prec)
      Rec_tnr.append(rec)
      Auc_tnr.append(auc)
      F1_tnr.append(f1)
      TPR_tnr.append(tpr)
      TNR_tnr.append(tnr)
      # performance with thresholf of best f1-score
      acc,prec,rec,f1,tpr, tnr, auc = label_by_score(df_test,max_f1_thr)

      Acc_f1.append(acc)
      Pre_f1.append(prec)
      Rec_f1.append(rec)
      Auc_f1.append(auc)
      F1_f1.append(f1)
      TPR_f1.append(tpr)
      TNR_f1.append(tnr)
      # performance with thresholf of best AUC (Area under the curve)
      acc,prec,rec,f1,tpr, tnr, auc = label_by_score(df_test,max_auc_thr)

      Acc_auc.append(acc)
      Pre_auc.append(prec)
      Rec_auc.append(rec)
      Auc_auc.append(auc)
      F1_auc.append(f1)
      TPR_auc.append(tpr)
      TNR_auc.append(tnr)

### Display Averaged Results

In [6]:
print("Best Acc")
print("accuracy",np.mean(Acc_a),np.std(Acc_a))
print("precision",np.mean(Pre_a),np.std(Pre_a))
print("recall",np.mean(Rec_a),np.std(Rec_a))
print("f1",np.mean(F1_a),np.std(F1_a))
print("AUC",np.mean(Auc_a),np.std(Auc_a))
print("TPR",np.mean(TPR_a),np.std(TPR_a))
print("TNR",np.mean(TNR_a),np.std(TNR_a))

Best Acc
accuracy 0.987817583008469 0.006040442084963935
precision 0.9673188405797102 0.03013635608350846
recall 0.9458333333333334 0.04187448175467038
f1 0.9555240858432349 0.023024568136091838
AUC 0.9702184987000424 0.02027303380302673
TPR 0.9458333333333334 0.04187448175467038
TNR 0.9946036640667513 0.005041802695827572


In [7]:
Accs

[0.32,
 0.36000000000000004,
 0.32,
 0.32,
 0.32,
 0.32,
 0.32,
 0.32,
 0.36000000000000004,
 0.32]

In [8]:
print("Best AUC")
print("accuracy",np.mean(Acc_auc),np.std(Acc_auc))
print("precision",np.mean(Pre_auc),np.std(Pre_auc))
print("recall",np.mean(Rec_auc),np.std(Rec_auc))
print("f1",np.mean(F1_auc),np.std(F1_auc))
print("AUC",np.mean(Auc_auc),np.std(Auc_auc))
print("TPR",np.mean(TPR_auc),np.std(TPR_auc))
print("TNR",np.mean(TNR_auc),np.std(TNR_auc))

Best AUC
accuracy 0.9785455034278803 0.011578667052637161
precision 0.8789981094174644 0.06255360878029179
recall 0.9875 0.019094065395649316
f1 0.9288536468200335 0.03656317938486012
AUC 0.9822975240341012 0.011460381325044657
TPR 0.9875 0.019094065395649316
TNR 0.9770950480682024 0.013166697374279113


In [9]:
AUCs

[0.12000000000000001,
 0.12000000000000001,
 0.12000000000000001,
 0.14,
 0.12000000000000001,
 0.12000000000000001,
 0.12000000000000001,
 0.12000000000000001,
 0.12000000000000001,
 0.12000000000000001]

In [10]:
print("Best Recall")
print("accuracy",np.mean(Acc_rec),np.std(Acc_rec))
print("precision",np.mean(Pre_rec),np.std(Pre_rec))
print("recall",np.mean(Rec_rec),np.std(Rec_rec))
print("f1",np.mean(F1_rec),np.std(F1_rec))
print("AUC",np.mean(Auc_rec),np.std(Auc_rec))
print("TPR",np.mean(TPR_rec),np.std(TPR_rec))
print("TNR",np.mean(TNR_rec),np.std(TNR_rec))

Best Recall
accuracy 0.9483599946229333 0.01723962561801241
precision 0.7387733045352214 0.06684858974694673
recall 0.9916666666666668 0.016666666666666653
f1 0.844819698720723 0.0434169173506848
AUC 0.9665071951145776 0.011270176553769499
TPR 0.9916666666666668 0.016666666666666653
TNR 0.9413477235624885 0.020611402625644402


In [11]:
Recalls

[0.02, 0.02, 0.12000000000000001, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02]

In [12]:
print("Best Precision")
print("accuracy",np.mean(Acc_prec),np.std(Acc_prec))
print("precision",np.mean(Pre_prec),np.std(Pre_prec))
print("recall",np.mean(Rec_prec),np.std(Rec_prec))
print("f1",np.mean(F1_prec),np.std(F1_prec))
print("AUC",np.mean(Auc_prec),np.std(Auc_prec))
print("TPR",np.mean(TPR_prec),np.std(TPR_prec))
print("TNR",np.mean(TNR_prec),np.std(TNR_prec))

Best Precision
accuracy 0.9675124344669982 0.01349816352293089
precision 0.9856926406926407 0.021878491938718395
recall 0.7791666666666666 0.10214708893442717
f1 0.8659503322262088 0.06479775485803481
AUC 0.888572087187859 0.05055712395523556
TPR 0.7791666666666666 0.10214708893442717
TNR 0.9979775077090514 0.0030894301926220275


In [13]:
Precisions

[0.8400000000000004,
 0.8400000000000004,
 0.8400000000000004,
 0.8400000000000004,
 0.8400000000000004,
 0.8400000000000004,
 0.8400000000000004,
 0.8400000000000004,
 0.8400000000000004,
 0.7000000000000003]

In [14]:
print("Best F1-score")
print("accuracy",np.mean(Acc_f1),np.std(Acc_f1))
print("precision",np.mean(Pre_f1),np.std(Pre_f1))
print("recall",np.mean(Rec_f1),np.std(Rec_f1))
print("f1",np.mean(F1_f1),np.std(F1_f1))
print("AUC",np.mean(Auc_f1),np.std(Auc_f1))
print("TPR",np.mean(TPR_f1),np.std(TPR_f1))
print("TNR",np.mean(TNR_f1),np.std(TNR_f1))

Best F1-score
accuracy 0.988977013039387 0.006051701473868545
precision 0.9673188405797102 0.03013635608350846
recall 0.9541666666666668 0.03461093276215865
f1 0.9600759878419453 0.022068976911978403
AUC 0.9743851653667089 0.017163675123497854
TPR 0.9541666666666668 0.03461093276215865
TNR 0.9946036640667513 0.005041802695827572


In [15]:
F1s

[0.32, 0.32, 0.32, 0.32, 0.32, 0.32, 0.32, 0.32, 0.32, 0.32]

In [16]:
print("Best TNR")
print("accuracy",np.mean(Acc_tnr),np.std(Acc_tnr))
print("precision",np.mean(Pre_tnr),np.std(Pre_tnr))
print("recall",np.mean(Rec_tnr),np.std(Rec_tnr))
print("f1",np.mean(F1_tnr),np.std(F1_tnr))
print("AUC",np.mean(Auc_tnr),np.std(Auc_tnr))
print("TPR",np.mean(TPR_tnr),np.std(TPR_tnr))
print("TNR",np.mean(TNR_tnr),np.std(TNR_tnr))

Best TNR
accuracy 0.9408119370883183 0.01772305405810052
precision 0.9878676470588236 0.02427862791338097
recall 0.5833333333333333 0.13043729868748774
f1 0.7239331977393595 0.10814346259908833
AUC 0.7909932583590302 0.06490951687178897
TPR 0.5833333333333333 0.13043729868748774
TNR 0.998653183384727 0.00269365231606793


In [17]:
TNRs

[0.9800000000000005,
 0.9800000000000005,
 0.9800000000000005,
 0.9800000000000005,
 0.9800000000000005,
 0.9800000000000005,
 0.9800000000000005,
 0.9800000000000005,
 0.9800000000000005,
 0.9800000000000005]